In [1]:
import pandas as pd
import numpy as np
import math 

In [2]:
# Data
lamination_data = pd.read_csv("https://raw.githubusercontent.com/yaswanthbandaru/transformer-design/main/DATA/EI-Laminations.csv")
swg_data = pd.read_csv("https://raw.githubusercontent.com/yaswanthbandaru/transformer-design/main/DATA/EMD%20-%20Sheet1.csv")


In [3]:
from autotransformer import AutoTransformer
from strip import Strip

In [4]:
# Initial variables for the calculation and validation
Frequency = 47 # Hz
Temperature_rise_goal = 30 # degree Celcius
Output_power = 2000 # watts
Efficiency = 95 # %
Input_voltage = 115 # volts
Ouptut_voltage = 115 # volts
Regulation = 5 # in a scale of 100
Bobbin_thickness = 1.5 
K_f = 4.44
K_u = 0.4
B_ac = 1.6
J = 235
insulation_thickness = 0.2 #mm

pi = np.pi #pi
a = 1.68 # coefficients for core loss
b = 1.86 # coefficients for core loss 
Rate_of_Cu = 950 # Rs / Kg
Rate_of_Fe = 250 # Rs / Kg
Resistivity_Cu =  1.68 * 10**-6 # ohm cm

## Algorithm

In [5]:
'''
Area = s * t * ww * wl

s, t, ww, wl muliples of 5

s * t * ww * wl  <= Area_product

50 < s < 400
50 < t < 250
20 < ww < 400
50 < wl < 700

s = [50, 55, 60 ... 445]
t = [50, 55, .. ]
'''



'\nArea = s * t * ww * wl\n\ns, t, ww, wl muliples of 5\n\ns * t * ww * wl  <= Area_product\n\n50 < s < 400\n50 < t < 250\n20 < ww < 400\n50 < wl < 700\n\ns = [50, 55, 60 ... 445]\nt = [50, 55, .. ]\n'

In [6]:
# for s in range(50,  400, 5):
#     for t in range(50, 250, 5):
#         for ww in range(20, 400, 5):
#             for wl in range(50, 700, 5):
#                 print(f"s * t * ww * wl = {s} * {t} * {ww} * {wl} = {s * t * ww * wl}")
                

## Strip function part

In [7]:
from strip_test import *

In [8]:
strip = Strip(
    frequency= 47 ,
    input_voltage= 230,
    output_voltage=230,
    output_power= 80000,
    efficiency= 96,
    regulation= 2,
    flux_density= 1.25,
    current_density= 300
    )

Apparent power : 163333.33333333334
Area product : 52179.839413881964
Input current : 347.82608695652175
a_wp : 115.94202898550725
a_ws : 115.94202898550725
primary swg : swg     SWG    mm CD Min mm CD Max mm  Normal Conductor Area mm²  \
0  10.0  3.24     3.218     3.284                    8.24479   

   Conductor Weight for 1000m/Kg  CR Normal  CR Max    CR Min  FC Min mm  \
0                         73.296    0.00203  0.00209  0.00198        NaN   

   FC Max mm  Medium Covering Min  Medium Covering Max  
0        NaN                0.086                 3.38  
secondary swg : swg     SWG    mm CD Min mm CD Max mm  Normal Conductor Area mm²  \
0  10.0  3.24     3.218     3.284                    8.24479   

   Conductor Weight for 1000m/Kg  CR Normal  CR Max    CR Min  FC Min mm  \
0                         73.296    0.00203  0.00209  0.00198        NaN   

   FC Max mm  Medium Covering Min  Medium Covering Max  
0        NaN                0.086                 3.38  


In [9]:
strip.total_cost(
    stack= 160,
    tongue= 145,
    winding_width= 35,
    winding_length= 200)

{'Area product': 162400000,
 'Stack mm': 160,
 'Tongue mm': 145,
 'wl mm': 200,
 'ww mm': 35,
 'Cu surface area': 71536.0,
 'Core surface area': 779300,
 'Temperature rise Cu': 106.5373685429474,
 'Temperature rise Fe': 0.2071997209526786,
 'Cost': 36241.03564416}

In [10]:
import numpy as np 
from scipy.optimize import minimize 

In [11]:
strip.apparent_power

163333.33333333334

In [12]:
strip.area_product

52179.839413881964

In [13]:
def objective(x):
    x1 = x[0] # stack
    x2 = x[1] # tongue
    x3 = x[2] # ww
    x4 = x[3] # wl
    return strip.cost(x1,x2,x3,x4)

def equality_constraint(x):
    return x[0]*x[1]*x[2]*x[3]-strip.area_product 

def constraint1(x):
    return x[0] - 50.0

def constraint2(x):
    return x[1] - 50.0

def constraint3(x):
    return x[2] - 20.0

def constraint4(x):
    return x[3] - 50.0 

In [14]:
np.zeros(4)

array([0., 0., 0., 0.])

In [15]:
# x0 = [50, 2, 10]
x0 = [50, 50, 30, 50]

print(objective(x0))
# print(equality_constraint(x0))

4000


In [16]:
b1 = (50.0, 400.0)
b2 = (50.0, 250.0)
b3 = (20.0, 400.0)
b4 = (50.0, 700.0)
bnds = (b1, b2, b3, b4) # bounds 
con1 = { 'type' : 'eq', 'fun': equality_constraint}
cons = [con1]


In [17]:
sol = minimize(objective, x0, method='SLSQP',bounds=bnds, constraints=cons)

In [18]:
for i in range(4):
    print(sol.x[i])

50.0
50.0
20.000000000013934
50.0
